<a href="https://colab.research.google.com/github/kjan318/Data-INSIGHTS-Lab/blob/main/People_Analytics_dashboard_Adv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
%%writefile requirements.txt
streamlit
pandas
plotly
Faker
scikit-learn
gspread
oauth2client
pyngrok

Overwriting requirements.txt


In [13]:
%%writefile google_sheets_handler.py
import gspread
import pandas as pd
from google.colab import userdata
from oauth2client.service_account import ServiceAccountCredentials
import os

def get_gspread_client():
    scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
    client = gspread.authorize(creds)
    return client

def get_spreadsheet():
    client = get_gspread_client()
    try:
        sheet_key = userdata.get('GGL_SHEET_KEY')
    except:
        sheet_key = os.environ.get('GGL_SHEET_KEY')
    return client.open_by_key(sheet_key)

def write_df_to_sheet_tab(spreadsheet, tab_name, df):
    """
    Writes a pandas DataFrame to a specific tab in a Google Sheet.
    It will create the tab if it doesn't exist, or clear it if it does.
    """
    try:
        worksheet = spreadsheet.worksheet(tab_name)
        # --- KEY STEP: This line deletes all existing data in the tab ---
        print(f"Found existing tab '{tab_name}'. Clearing all data before writing...")
        worksheet.clear()
    except gspread.WorksheetNotFound:
        print(f"Creating new tab: '{tab_name}'...")
        worksheet = spreadsheet.add_worksheet(title=tab_name, rows="1", cols="1")

    # Write the new data to the now-empty sheet
    worksheet.update([df.columns.values.tolist()] + df.astype(str).values.tolist())
    print(f"Successfully wrote new data to tab: '{tab_name}'")

def read_sheet_tab_to_df(spreadsheet, tab_name):
    worksheet = spreadsheet.worksheet(tab_name)
    data = worksheet.get_all_records()
    return pd.DataFrame(data)



Overwriting google_sheets_handler.py


In [15]:
#%%writefile initial_data_generator.py
import pandas as pd
from faker import Faker
import random
from datetime import date, timedelta
import google_sheets_handler as gsh

def generate_raw_data(num_employees=250):
    print("Starting raw data generation with complete schema...")
    fake = Faker()

    employees_data = []
    for i in range(num_employees):
        emp_id = 1000 + i
        hire_date = fake.date_between(start_date='-4y', end_date='today')
        is_terminated = random.random() < 0.25 # 25% of employees have a termination date
        term_date = fake.date_between(start_date=hire_date, end_date='today') if is_terminated else None

        employees_data.append({
            'Employee_ID': emp_id,
            'Department': random.choice(['Engineering', 'Sales', 'HR', 'Marketing']),
            'Hire_Date': hire_date,
            'Termination_Date': term_date, # This is the missing column
            'Performance_Rating': random.randint(1, 5)
        })

    performance_data = []
    for record in employees_data:
        performance_data.append({
            'Employee_ID': record['Employee_ID'],
            'Performance_Cycle_Date': fake.date_between(start_date=record['Hire_Date'], end_date='today'),
            'Performance_Rating': record['Performance_Rating']
        })

    survey_data = []
    for record in employees_data:
        survey_data.append({
            'Employee_ID': record['Employee_ID'],
            'Survey_Date': fake.date_between(start_date=record['Hire_Date'], end_date='today'),
            'Survey_Type': 'eNPS',
            'Score': random.randint(0, 10)
        })

    df_employees = pd.DataFrame(employees_data)
    df_performance = pd.DataFrame(performance_data)
    df_surveys = pd.DataFrame(survey_data)

    spreadsheet = gsh.get_spreadsheet()
    gsh.write_df_to_sheet_tab(spreadsheet, "raw_employees", df_employees)
    gsh.write_df_to_sheet_tab(spreadsheet, "raw_performance", df_performance)
    gsh.write_df_to_sheet_tab(spreadsheet, "raw_surveys", df_surveys)
    print("✅ Raw data generation complete and uploaded to Google Sheet.")

if __name__ == "__main__":
    generate_raw_data()

Starting raw data generation with complete schema...
Successfully wrote data to tab: 'raw_employees'
Successfully wrote data to tab: 'raw_performance'
Successfully wrote data to tab: 'raw_surveys'
✅ Raw data generation complete and uploaded to Google Sheet.


streamlit
pandas
plotly
gspread
gspread-dataframe
oauth2client
Faker

In [19]:
#%%writefile model_trainer.py
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import joblib
import google_sheets_handler as gsh

def process_data_and_train():
    print("Reading raw data from Google Sheet...")
    spreadsheet = gsh.get_spreadsheet()
    df_employees = gsh.read_sheet_tab_to_df(spreadsheet, "raw_employees")
    df_performance = gsh.read_sheet_tab_to_df(spreadsheet, "raw_performance")
    df_surveys = gsh.read_sheet_tab_to_df(spreadsheet, "raw_surveys")

    # Ensure consistent data types for the merge key
    df_employees['Employee_ID'] = pd.to_numeric(df_employees['Employee_ID'], errors='coerce')
    df_performance['Employee_ID'] = pd.to_numeric(df_performance['Employee_ID'], errors='coerce')
    df_surveys['Employee_ID'] = pd.to_numeric(df_surveys['Employee_ID'], errors='coerce')

    # Drop rows with invalid Employee_ID after coercion
    df_employees.dropna(subset=['Employee_ID'], inplace=True)
    df_performance.dropna(subset=['Employee_ID'], inplace=True)
    df_surveys.dropna(subset=['Employee_ID'], inplace=True)

    # Convert other types
    df_employees['Hire_Date'] = pd.to_datetime(df_employees['Hire_Date'], errors='coerce')
    df_employees['Termination_Date'] = pd.to_datetime(df_employees['Termination_Date'], errors='coerce')
    df_performance['Performance_Rating'] = pd.to_numeric(df_performance['Performance_Rating'], errors='coerce')
    df_surveys['Score'] = pd.to_numeric(df_surveys['Score'], errors='coerce')

    # Create Tidy Table
    # Ensure we get the latest performance for each employee
    latest_performance_indices = df_performance.groupby('Employee_ID')['Performance_Cycle_Date'].idxmax().dropna()
    latest_performance = df_performance.loc[latest_performance_indices].copy()


    # Ensure we get the latest eNPS for each employee
    latest_enps_indices = df_surveys[df_surveys['Survey_Type'] == 'eNPS'].groupby('Employee_ID')['Survey_Date'].idxmax().dropna()
    latest_enps = df_surveys.loc[latest_enps_indices].copy()


    # Merge the dataframes - merge the full latest performance and enps dataframes
    tidy_df = pd.merge(df_employees, latest_performance, on='Employee_ID', how='left')
    tidy_df = pd.merge(tidy_df, latest_enps, on='Employee_ID', how='left')

    # Rename columns after merge to ensure correct names
    tidy_df.rename(columns={'Performance_Rating_y': 'Performance_Rating', 'Score': 'eNPS_Score'}, inplace=True)
    # Drop redundant columns after merge if they exist
    tidy_df.drop(columns=['Performance_Rating_x', 'Performance_Cycle_Date', 'Survey_Date', 'Survey_Type'], errors='ignore', inplace=True)


    tidy_df['Tenure'] = (pd.to_datetime('today') - tidy_df['Hire_Date']).dt.days / 365.25
    tidy_df['Turnover'] = tidy_df['Termination_Date'].notna().astype(int)

    print("Writing Tidy Master Table back to Google Sheet...")
    gsh.write_df_to_sheet_tab(spreadsheet, "Tidy_Master_Table", tidy_df)

    print("Training turnover prediction model...")
    features = ['Tenure', 'Performance_Rating', 'eNPS_Score']

    # Ensure features exist in the dataframe before selecting
    missing_features = [f for f in features if f not in tidy_df.columns]
    if missing_features:
        print(f"Warning: Missing features in tidy_df: {missing_features}. Model training may fail or be inaccurate.")
        # As a temporary fix, drop missing features from the list
        features = [f for f in features if f in tidy_df.columns]
        if not features:
            print("Error: No valid features remaining for model training.")
            return # Exit if no features are available


    X = tidy_df[features].fillna(tidy_df[features].median())
    y = tidy_df['Turnover']
    model = RandomForestClassifier(random_state=42).fit(X, y)

    joblib.dump(model, 'turnover_model.pkl')
    print("✅ Model trained and saved as turnover_model.pkl")

if __name__ == "__main__":
    process_data_and_train()

Reading raw data from Google Sheet...


/tmp/ipython-input-3017315193.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_employees['Termination_Date'] = pd.to_datetime(df_employees['Termination_Date'], errors='coerce')


Writing Tidy Master Table back to Google Sheet...
Successfully wrote data to tab: 'Tidy_Master_Table'
Training turnover prediction model...
✅ Model trained and saved as turnover_model.pkl


In [20]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px
import joblib
import os
import google_sheets_handler as gsh

st.set_page_config(page_title="People Analytics Dashboard", page_icon="🚀", layout="wide")

@st.cache_data(ttl=600)
def load_data():
    spreadsheet = gsh.get_spreadsheet()
    tidy_df = gsh.read_sheet_tab_to_df(spreadsheet, "Tidy_Master_Table")
    # Convert types after reading from sheets
    for col in ['Hire_Date', 'Termination_Date']:
        tidy_df[col] = pd.to_datetime(tidy_df[col], errors='coerce')
    for col in ['Performance_Rating', 'eNPS_Score', 'Tenure']:
        tidy_df[col] = pd.to_numeric(tidy_df[col], errors='coerce')
    return tidy_df

tidy_df = load_data()
model = joblib.load('turnover_model.pkl')

# --- Sidebar Filters ---
st.sidebar.header("Global Filters")
selected_dept = st.sidebar.multiselect("Department", options=tidy_df['Department'].unique(), default=tidy_df['Department'].unique())
filtered_df = tidy_df[tidy_df['Department'].isin(selected_dept)]

# --- Main Dashboard ---
st.title("🚀 People Analytics Dashboard")

# --- KPI & Analytics Tabs ---
tab1, tab2, tab3 = st.tabs(["📊 KPI Dashboard", "🔮 Predictive Insights", "💡 Prescriptive Actions"])

with tab1:
    st.header("Key Performance Indicators")
    col1, col2, col3 = st.columns(3)
    # Lagging KPI
    turnover_rate = (filtered_df['Termination_Date'].notna().sum() / len(filtered_df)) * 100
    col1.metric("Overall Turnover Rate (Lagging)", f"{turnover_rate:.2f}%")
    # Leading KPI
    avg_enps = filtered_df['eNPS_Score'].mean()
    col2.metric("Average eNPS (Leading)", f"{avg_enps:.2f}")
    # Descriptive KPI
    headcount = len(filtered_df[filtered_df['Termination_Date'].isna()])
    col3.metric("Current Headcount (Descriptive)", headcount)

    # Add more KPI visuals here...
    fig = px.histogram(filtered_df, x='Tenure', title='Employee Tenure Distribution')
    st.plotly_chart(fig, use_container_width=True)

with tab2:
    st.header("Predictive Analytics: Employee Turnover Risk")
    current_employees = filtered_df[filtered_df['Termination_Date'].isna()].copy()
    features = ['Tenure', 'Performance_Rating', 'eNPS_Score']
    current_employees['Turnover_Risk'] = model.predict_proba(current_employees[features].fillna(current_employees[features].median()))[:, 1]

    st.subheader("Top Employees at Risk")
    st.dataframe(current_employees[['Employee_ID', 'Department', 'Tenure', 'Turnover_Risk']].sort_values('Turnover_Risk', ascending=False).head())

with tab3:
    st.header("Prescriptive Actions")
    high_risk_employee = current_employees.sort_values('Turnover_Risk', ascending=False).iloc[0]
    st.subheader(f"Recommended Actions for Employee ID: {high_risk_employee['Employee_ID']}")
    if high_risk_employee['Performance_Rating'] >= 4:
        st.warning("High-performer at risk! **Action:** Schedule a career development discussion with their manager immediately.")
    if high_risk_employee['eNPS_Score'] <= 6:
        st.info("Low engagement score detected. **Action:** Manager to conduct a 'stay interview' to understand concerns.")

Overwriting app.py


In [25]:
!pip install -r requirements.txt

In [26]:
from pyngrok import ngrok
from google.colab import userdata
import os

os.environ['GGL_SHEET_KEY'] = userdata.get('GGL_SHEET_KEY')

ngrok.kill()
NGROK_AUTH_TOKEN = "31fq6Ze7AbyVhGp5cFN3WM5HSPY_5dwBEa7toz8xUPZ9S5aPe"  # <--- PASTE YOUR TOKEN HERE
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(8501)
print(f"🚀 Your dashboard is live at: {public_url}")
!streamlit run app.py --server.port 8501 --server.headless true

🚀 Your dashboard is live at: NgrokTunnel: "https://16ab8c03944e.ngrok-free.app" -> "http://localhost:8501"





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.139.212.246:8501



  Stopping...
  Stopping...


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Backup

In [ ]:
%%writefile app.py
# This is the same app.py file from the original response.
# No changes are needed here.
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import date, timedelta
from google_sheets_handler import get_sheet, read_data_from_sheet, append_data_to_sheet
from data_generator import generate_employee_data

# --- Page Configuration ---
st.set_page_config(
    page_title="People Analytics Dashboard",
    page_icon="📊",
    layout="wide",
)

# --- Data Loading and Caching ---
# @st.cache_data decorator removed for Colab compatibility with gspread objects
def load_data():
    sheet = get_sheet()
    df = read_data_from_sheet(sheet)
    if df.empty:
        st.warning("Data sheet is empty. Generating initial data for the last 12 months.")
        today = date.today()
        initial_data = generate_employee_data(start_date=today - timedelta(days=365), end_date=today)
        append_data_to_sheet(sheet, initial_data)
        df = read_data_from_sheet(sheet) # Reload data
    return df

df = load_data()

# --- Sidebar Filters ---
st.sidebar.header("📊 People Analytics Dashboard")
st.sidebar.markdown("Filter your data to get specific insights.")

# Ensure Hire_Date is datetime before finding min/max
df["Hire_Date"] = pd.to_datetime(df["Hire_Date"])

min_date = df["Hire_Date"].min().date()
max_date = df["Hire_Date"].max().date()

date_range = st.sidebar.date_input(
    "Select Hire Date Range",
    value=(min_date, max_date),
    min_value=min_date,
    max_value=max_date,
)

selected_departments = st.sidebar.multiselect(
    "Select Departments",
    options=sorted(df["Department"].unique()),
    default=sorted(df["Department"].unique()),
)

selected_locations = st.sidebar.multiselect(
    "Select Locations",
    options=sorted(df["Location"].unique()),
    default=sorted(df["Location"].unique()),
)

# --- Data Generation Sidebar ---
st.sidebar.markdown("---")
st.sidebar.header("Generate New Data")
if st.sidebar.button("Generate New Month's Data"):
    with st.spinner("Generating and appending new data..."):
        last_date = df["Hire_Date"].max().date()
        new_data = generate_employee_data(start_date=last_date + timedelta(days=1), end_date=last_date + timedelta(days=31))
        sheet = get_sheet()
        append_data_to_sheet(sheet, new_data)
        st.success("New data generated! Please refresh the page to see updates.")


# --- Filter Data based on selection ---
start_date, end_date = date_range
filtered_df = df[
    (df["Hire_Date"].dt.date >= start_date) &
    (df["Hire_Date"].dt.date <= end_date) &
    (df["Department"].isin(selected_departments)) &
    (df["Location"].isin(selected_locations))
]

# --- Main Dashboard Display ---
st.title("📈 HR KPI Dashboard")
st.markdown("This dashboard provides an overview of key human resources metrics.")

if filtered_df.empty:
    st.warning("No data available for the selected filters.")
else:
    # --- Key Metrics ---
    total_hires = len(filtered_df)

    # Ensure Termination_Date is also datetime
    filtered_df['Termination_Date'] = pd.to_datetime(filtered_df['Termination_Date'])
    turnover_90_day = filtered_df[
        (filtered_df['Termination_Date'].notna()) &
        ((filtered_df['Termination_Date'] - filtered_df['Hire_Date']).dt.days <= 90)
    ].shape[0]
    turnover_rate = (turnover_90_day / total_hires * 100) if total_hires > 0 else 0
    offer_acceptance_rate = (filtered_df[filtered_df['Offer_Status'] == 'Accepted'].shape[0] / filtered_df.shape[0] * 100)

    col1, col2, col3 = st.columns(3)
    with col1:
        st.metric(label="Total Hires", value=f"{total_hires}")
    with col2:
        st.metric(label="90-Day Turnover Rate", value=f"{turnover_rate:.2f}%")
    with col3:
        st.metric(label="Offer Acceptance Rate", value=f"{offer_acceptance_rate:.2f}%")

    st.markdown("---")

    # --- Visualizations ---
    with st.container():
        st.header("KPI Deep Dive")

        with st.expander("Attract & Hire"):
            col1, col2 = st.columns(2)
            with col1:
                filtered_df['Requisition_Approval_Date'] = pd.to_datetime(filtered_df['Requisition_Approval_Date'])
                filtered_df['Offer_Acceptance_Date'] = pd.to_datetime(filtered_df['Offer_Acceptance_Date'])
                filtered_df['Time_to_Fill'] = (filtered_df['Offer_Acceptance_Date'] - filtered_df['Requisition_Approval_Date']).dt.days
                avg_time_to_fill = filtered_df.groupby(pd.Grouper(key='Hire_Date', freq='M'))['Time_to_Fill'].mean().reset_index()

                fig = px.line(avg_time_to_fill, x='Hire_Date', y='Time_to_Fill', title='Average Time to Fill (Days)', markers=True)
                st.plotly_chart(fig, use_container_width=True)
                st.markdown("_A line chart shows the trend in hiring efficiency over time._")

            with col2:
                source_counts = filtered_df['Application_Source_Channel'].value_counts().reset_index()
                source_counts.columns = ['Source', 'Count']

                fig = px.pie(source_counts, names='Source', values='Count', title='Source of Hire Effectiveness')
                st.plotly_chart(fig, use_container_width=True)
                st.markdown("_A pie chart is used to show the proportion of hires from each recruitment channel._")

        with st.expander("Talent & Leadership"):
            turnover_df = filtered_df[filtered_df['Termination_Date'].notna()]
            if not turnover_df.empty:
                turnover_by_dept = turnover_df.groupby('Department').size().reset_index(name='Turnover Count')
                fig = px.bar(turnover_by_dept, x='Department', y='Turnover Count', title='Turnover by Department')
                st.plotly_chart(fig, use_container_width=True)
                st.markdown("_A bar chart helps compare turnover volumes across different departments._")
            else:
                st.info("No turnover data to display for the selected period.")